# Content-based recommendations

In [1]:
import pandas as pd
import sklearn as sk
import json

In [4]:
#import the original dataset provided by the AI-research group
df_stu = pd.read_excel(r"Files/Studiekeuze123/Download Studiekeuze123.xlsx",sheet_name='Studies')
df_opl = pd.read_excel(r"Files/Studiekeuze123/Download Studiekeuze123.xlsx",sheet_name='Opleidingen')
df_ber = pd.read_excel(r"Files/Studiekeuze123/Download Studiekeuze123.xlsx",sheet_name='Beroepen')

In [5]:
df_map_opl = df_opl[df_opl.columns[[3, 4, 5, 6, 7, 11, 12, 13, 14, 16, 17, 18, 20, 21, 27, 29, 30,31, 32, 33, 34, 35, 42, 43, 45, 46, 48, 50, 57, 58 ]]]
df_map_stu = df_stu[df_stu.columns[[0, 1, 2, 3, 4, 5, 6, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22 ]]]


In [6]:
### The NSE destinguishes the following ratings. These have been renamed using the documentation, to simplify the interpretation. ###
df_map_stu = df_map_stu.rename({'AlgemeenOordeel01Wx': 'Rating_gem', 
                                'nAlgemeenOordeel01' :'n_students_rating', 
                                'AlgemeenOordeel01aWPrc': 'Percentage_zeer_ontevreden', 
                                'AlgemeenOordeel01bWPrc': 'Percentage_ontevreden', 
                                'AlgemeenOordeel01cWPrc': 'Percentage_gemiddeld', 
                                'AlgemeenOordeel01dWPrc': 'Percentage_tevreden', 
                                'AlgemeenOordeel01eWPrc': 'Percentage_zeer_tevreden'
                               }, axis=1)  # new method

In [7]:
### Round the float to 2 decimals, to enhance the speed of the recommendations ###
df_map_stu['Rating_gem'] = df_map_stu['Rating_gem'].round(2)
df_map_stu['Percentage_zeer_ontevreden'] = df_map_stu['Percentage_zeer_ontevreden'].round(2)
df_map_stu['Percentage_ontevreden'] = df_map_stu['Percentage_ontevreden'].round(2)
df_map_stu['Percentage_gemiddeld'] = df_map_stu['Percentage_gemiddeld'].round(2)
df_map_stu['Percentage_tevreden'] = df_map_stu['Percentage_tevreden'].round(2)
df_map_stu['Percentage_zeer_tevreden'] = df_map_stu['Percentage_zeer_tevreden'].round(2)

In [8]:
df_map_stu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304 entries, 0 to 1303
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   OpleidingscodeSamen         1304 non-null   int64  
 1   NaamOpleiding               1304 non-null   object 
 2   Studiebeschrijving          1150 non-null   object 
 3   Sector                      1304 non-null   object 
 4   SoortOpleiding              1304 non-null   object 
 5   SoortHo                     1304 non-null   object 
 6   PrognoseTekst               598 non-null    object 
 7   NSEEnqueteJaar              971 non-null    float64
 8   Rating_gem                  926 non-null    float64
 9   n_students_rating           926 non-null    float64
 10  Percentage_zeer_ontevreden  926 non-null    float64
 11  Percentage_ontevreden       926 non-null    float64
 12  Percentage_gemiddeld        926 non-null    float64
 13  Percentage_tevreden         926 n

## Content based

In [9]:
# Import numpy and the Jaccard similarity function
import numpy as np
from sklearn.metrics import jaccard_score

# Import functions from scipy
from scipy.spatial.distance import pdist, squareform

In [10]:
df_stu_content = df_map_stu[df_map_stu.columns[[0, 1, 2, 3]]]

In [11]:
df_stu_content.head()

,OpleidingscodeSamen,NaamOpleiding,Studiebeschrijving,Sector
0,60449,Aansprakelijkheid en verzekering,De universitaire masterstudie Aansprakelijkhei...,Recht en Bestuur
1,50668,Aarde en Economie,De universitaire bachelorstudie Aarde en Econo...,Interdisciplinair
2,56986,Aardwetenschappen,De universitaire bachelorstudie Aardwetenschap...,Aarde en Milieu
3,34406,Accountancy,De hbo bachelorstudie Accountancy is gericht o...,Economie en Bedrijf
4,56435,Accountancy,De universitaire bachelorstudie Accountancy is...,Economie en Bedrijf


In [12]:
df_stu_content = df_stu_content.dropna()
df_stu_content = df_stu_content.drop_duplicates(subset='NaamOpleiding', keep='first', inplace=False) # some courses of study have been duplicated in de the dataset

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=2, max_df=0.7)
vectorized_data = vectorizer.fit_transform(df_stu_content['Studiebeschrijving'])

tfidf_df = pd.DataFrame(vectorized_data.toarray(), columns=vectorizer.get_feature_names())

# Assign the movie titles to the index and inspect
tfidf_df.index = df_stu_content['NaamOpleiding']
tfidf_df.head()

,21e,2e,3d,aan,aandacht,aandoening,aangeboden,aangereikt,aangevuld,aanleg,...,zowel,zuid,zuidoost,zul,zullen,zult,zwang,zwijgen,én,één
NaamOpleiding,,,,,,,,,,,,,,,,,,,,,
Aansprakelijkheid en verzekering,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aarde en Economie,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aardwetenschappen,0.0,0.0,0.0,0.058591,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Accountancy,0.0,0.0,0.0,0.151994,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Accountancy and Control,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Import cosine_similarity measure
from sklearn.metrics.pairwise import cosine_similarity

# Create the array of cosine similarity values
cosine_similarity_array = cosine_similarity(tfidf_df)

# Wrap the array in a pandas DataFrame
cosine_similarity_df = pd.DataFrame(cosine_similarity_array, index=tfidf_df.index, columns=tfidf_df.index)

# Print the top 5 rows of the DataFrame
cosine_similarity_df.head(5)

NaamOpleiding,Aansprakelijkheid en verzekering,Aarde en Economie,Aardwetenschappen,Accountancy,Accountancy and Control,Accountancy and Controlling,Accounting and Control,"Accounting, Auditing and Control",Actuarial Science and Mathematical Finance,Actuariële Wetenschappen,...,Watermanagement,Welzijn en zorg voor ouderen,Werktuigbouwkunde,Wiskunde,"Work, Health and Career",Zorg en Technologie,Zorgethiek en Beleid,Zorgmanagement,Zorgtraject Ontwerp,Zuid- en Zuidoost-Aziëstudies
NaamOpleiding,,,,,,,,,,,,,,,,,,,,,
Aansprakelijkheid en verzekering,1.000000,0.007013,0.009654,0.000000,0.019748,0.017079,0.019851,0.019300,0.018489,0.013456,...,0.008921,0.011736,0.038540,0.050479,0.053823,0.024677,0.024638,0.017489,0.010571,0.047876
Aarde en Economie,0.007013,1.000000,0.160249,0.007534,0.031329,0.037057,0.031493,0.031159,0.087213,0.047693,...,0.070000,0.043872,0.090773,0.053894,0.038558,0.000000,0.169056,0.048987,0.015087,0.030594
Aardwetenschappen,0.009654,0.160249,1.000000,0.039565,0.018109,0.012211,0.014193,0.019592,0.034428,0.044349,...,0.061181,0.010535,0.073073,0.054520,0.028862,0.046993,0.032103,0.042418,0.035114,0.014151
Accountancy,0.000000,0.007534,0.039565,1.000000,0.228571,0.244090,0.177401,0.110491,0.089088,0.030136,...,0.042543,0.009508,0.076233,0.013161,0.028552,0.071668,0.002985,0.056205,0.025136,0.045646
Accountancy and Control,0.019748,0.031329,0.018109,0.228571,1.000000,0.724854,0.945038,0.177804,0.169448,0.082998,...,0.033944,0.010726,0.065248,0.015039,0.032184,0.005479,0.040830,0.028459,0.021891,0.006561


In [15]:
### Generate the user profile by finding the average TF-IDF scores of each of the features of the experiences ###
list_of_positive_experiences = ['Communication and Multimedia Design', 'Creative Business', 'Farmakunde']

# Create a subset of only the movies the user has enjoyed
experiences_enjoyed_df = tfidf_df.reindex(list_of_positive_experiences)

# Generate the user profile by finding the average scores of movies they enjoyed
user_prof = experiences_enjoyed_df.mean()

# Inspect the results
print('The words most common for your experiences:')

print(user_prof.sort_values(ascending = False))

The words most common for your experiences:
interactieve      0.216482
farmaceutische    0.216305
media             0.190901
multimedia        0.142391
taken             0.118791
                    ...   
gesteld           0.000000
getraind          0.000000
geven             0.000000
gevestigde        0.000000
één               0.000000
Length: 3059, dtype: float64


In [16]:
# Find subset of tfidf_df that does not include movies in list_of_movies_enjoyed
tfidf_subset_df = tfidf_df.drop(list_of_positive_experiences, axis=0)

# Calculate the cosine_similarity and wrap it in a DataFrame
similarity_array = cosine_similarity(user_prof.values.reshape(1, -1), tfidf_subset_df)
similarity_df = pd.DataFrame(similarity_array.T, index=tfidf_subset_df.index, columns=["similarity_score"])

# Sort the values from high to low by the values in the similarity_score
sorted_similarity_df = similarity_df.sort_values(by="similarity_score", ascending=False)

# Inspect the most similar to the user preferences
print(sorted_similarity_df.head(10))

                                                    similarity_score
NaamOpleiding                                                       
Type and Media                                              0.333844
Creative Media and Game Technologies                        0.288826
Farmaceutisch Consulent                                     0.238690
Geo Media & Design                                          0.215672
International Bachelor's Programme in Communica...          0.187549
Mediastudies (research)                                     0.185604
Kunst en Economie                                           0.181729
Film en Televisie                                           0.181330
Communication Science (research)                            0.178420
Crossmediale Communicatie                                   0.173709
